# <b>DNaseI-Seq workshop part 3 - Downstream analysis</b>

In the final part of the workshopwe will carry out a downstream analysis of the differential peak sets identified in workshop part 2. Here we will learn how to do
* Motif enrichment analysis with HOMER
* Peak annotation with HOMER
* Creating read density plots with DeepTools
* Creating average profiles with DeepTools

## <b>Step 0. Preparation</b>

Load the required software into your bluebear session

In [ ]:
module load HOMER/4.11-foss-2019b
module load deepTools/3.5.0-foss-2020a-Python-3.8.2

## <b>Step 1. Motif enrichment with HOMER</b>

* <b>H</b>ypergeometric <b>O</b>ptimization of <b>M</b>otif <b>E</b>n<b>R</b>ichment (HOMER) is a suite of tools for motif discovery as well as annotation of peaks in a bed file.
* HOMER is a widely used and cited tool which is used for any kind of analysis that requires a BED file (DNaseI, ATAC, ChIP, HiC etc...)
* The only input files that we need for this are the two BED files of NAg and TAg specific peaks

Here we will use the <b>findMotifsGenome.pl</b> function in HOMER

In [ ]:
findMotifsGenome.pl NAg_specific_sites.bed mm10 NAg_specific_sites_motifs -size 200 -noknown -preparsedDir ../data_for_workshop/mm10_preparsed
findMotifsGenome.pl TAg_specific_sites.bed mm10 NAg_specific_sites_motifs -size 200 -noknown -preparsedDir ../data_for_workshop/mm10_preparsed

This is what each part of these commands do:
* <span style="color:#0000DD">NAg_specific_sites.bed</span> - The BED file of sites you want to search for motifs in
* <span style="color:#0000DD">mm10</span> - The genome version we used to align the data - We used mm10 but for human you might use hg38
* <span style="color:#0000DD">NAg_specific_sites_motifs</span> - The name of the folder to save the results in
* <span style="color:#0000DD">-size 200</span> - The size of peaks in which to search for motifs. This will tell Homer to re-size all peaks to 200bp centered on the peak center
* <span style="color:#0000DD">-noknown</span> - Do not search for known motifs. This will tell Homer to only do a de-novo motif search
* <span style="color:#0000DD">-preparsedDir ../data_for_workshop/mm10_preparsed</span> - Provide the path to where the pre-prepared genome files are

Once Homer is finished you can download the files from the output directory and open the .html file in your web browser

## <b>Step 2. Peak annotation with HOMER</b>

Here we will use the <b>annotatePeaks.pl</b> function in HOMER to learn a bit more about the peaks we have called. This information includes:
* The name of the nearest gene
* The distance from the TSS of the nearest gene. This can be used to determine if a peak is in the gene promoter or in a distal element
* Where the peak is located in relation to the gene (Intergenic, Intragenic, Intronic, Promoter etc.)

In [ ]:
annotatePeaks.pl NAg_specific_sites.bed mm10 > NAg_specific_sites_annotated.tsv
annotatePeaks.pl TAg_specific_sites.bed mm10 > TAg_specific_sites_annotated.tsv

This is what each part of these commands do:
* <span style="color:#0000DD">NAg_specific_sites.bed</span> - The BED file of sites you want to annotate
* <span style="color:#0000DD">mm10</span> - The genome version we used to align the data - We used mm10 but for human you might use hg38
* <span style="color:#0000DD">> NAg_specific_sites_annotated.tsv</span> - The output file

The output file is a tab-delimited file that can be opened in programs like R or Excel. Dowload this file and have a look

## <b>Step 3. Creating read density plots with DeepTools</b>

* First, we will need to convert a BAM alignment file into the bigWig format. 
* This format will contain the number of reads aligned to each position in the genome. 
* It could also be loaded into genome browser programs such as IGV if you want to have a look at the peaks

In this workshop you will only make the bigWig for one of the example files from part 1. To save time bigWig files for the full dataset have already been prepared. Here we will use the <b>bamCoverage</b> function in <b>DeepTools</b>

In [ ]:
bamCoverage -b NAg_Rep1_sample_rmdup.bam -o NAg_Rep1_sample_rmdup.bw --normalizeUsing RPKM --scaleFactor 10 --blackListFileName /rds/projects/k/knowletj-mibtp-masterclasses/MITBP_Masterclass_2021/data_for_workshop/Annotation/mm10-blacklist.v2.bed



Here is what these options do:
* <span style="color:#0000DD">-b NAg_Rep1_sample_rmdup.bam</span> - The input file (a BAM file). Here we use the PCR de-duplicated alignment from part 1
* <span style="color:#0000DD">-o NAg_Rep1_sample_rmdup.bw</span> - The Output file. A bigWig file
* <span style="color:#0000DD">--normalizeUsing RPKM</span> - Normalize the read counts as Reads Per Kilobase per Million mapped reads (RPKM). Other options inluce CPM, BPM, RPGC
* <span style="color:#0000DD">--scaleFactor 10</span> - Multiply all values in the bigWig by this value (optional)
* <span style="color:#0000DD">--blackListFileName</span> - Provide the path to the mm10 blacklist. Alignments in these regions will be excluded from the bigWig

Now we can create the density plot

* Read density plots (such as that shown in figure 2C of Bevington et al.) are useful to compare the distribution of peaks across different samples
* To make this we will need a bigWig file for each of the samples as well as a BED file of peaks ordered by their fold-difference between NAg and TAg
* For this we will use the file <b>TAgNAg_ordered.bed</b> we made in part 2 of this workshop
* We will then use the <b>computeMatix</b> and <b>plotHeatmap</b> functions in <b>DeepTools</b>

In [ ]:
computeMatrix reference-point --referencePoint center -o TAgNAg_density_plot.matrix -a 1000 -b 1000 -R TAgNAg_ordered.bed -S ../data_for_workshop/bigWig/NAg_Rep1.bw ../data_for_workshop/bigWig/NAg_Rep2.bw ../data_for_workshop/bigWig/TAg_Rep1.bw ../data_for_workshop/bigWig/TAg_Rep2.bw



Here is what each of these options do:
* <span style="color:#0000DD">reference-point --referencePoint center</span> - Tells DeepTools to use center the density plot on the center of the peak
* <span style="color:#0000DD">-o TAgNAg_density_plot.matrix</span> - The output file. Will be used to plot the heatmap
* <span style="color:#0000DD">-a 1000 -b 1000</span> - The size of the region to plot. This specifies 1000 bases before peak center (-b) and 1000 bases after the peak (-a)
* <span style="color:#0000DD">-R TAgNAg_ordered.bed</span> - The BED file of ordered sites to plot
* <span style="color:#0000DD">-S</span> - The sequence files - These are the bigWig files for each of samples we want to plot

Finally, we can plot the heatmap.

In [ ]:
plotHeatmap -m TAgNAg_density_plot.matrix --outFileName TAgNAg_density_plot.pdf --colorList 'white,red' --sortRegions no --zMax 25 --whatToShow 'heatmap only' --missingDataColor 'white'


Here is what each of these options do:
* <span style="color:#0000DD">-m TAgNAg_density_plot.matrix</span> - The input file which is the matrix file we made with computeMatrix
* <span style="color:#0000DD">--outFileName TAgNAg_desnity_plot.pdf</span> - The output file name. Here DeepTools will create a pdf file for the plot. It can also make png, svg, eps, tiff etc.
* <span style="color:#0000DD">--colorList 'white,red'</span> - What colors to use for the heatmap
* <span style="color:#0000DD">--sortRegions no</span> - By default DeepTools will re-sort the peaks by their maximum peak height. We have already sorted the files by their fold-changes, so we can tell DeepTools not to re-sort here
* <span style="color:#0000DD">--zMax 25</span> - The maximum value to show on the heatmap. Any value above this will be re-set to 25
* <span style="color:#0000DD">--whatToShow 'heatmap only'</span> - Only plot the heatmap. By default it will also plot an average profile above the heatmap
* <span style="color:#0000DD">--missingDataColor 'white'</span> - What color to use with missing values in the heatmap. These could be for example the blacklisted regions we excluded

## <b>Step 4. Creating Average profiles with DeepTools<b>

The density plots above are useful to show a genome-wide view of the DHSs in a single image. However, we may also want to show the differential sites in more detail. In the final part of this workshop we will create an average profile of the differentially accessible peaks.

To do this we will once again use the <b>computeMatrix</b> function from <b>DeepTools</b> follwed by the <b>plotProfile</b> function

In [ ]:
computeMatrix reference-point --referencePoint center -o NAg_specific_sites_AvgProfile.matrix -a 1000 -b 1000 -R NAg_specific_sites.bed -S ../data_for_workshop/bigWig/NAg_Rep1.bw ../data_for_workshop/bigWig/NAg_Rep2.bw ../data_for_workshop/bigWig/TAg_Rep1.bw ../data_for_workshop/bigWig/TAg_Rep2.bw
computeMatrix reference-point --referencePoint center -o TAg_specific_sites_AvgProfile.matrix -a 1000 -b 1000 -R TAg_specific_sites.bed -S ../data_for_workshop/bigWig/NAg_Rep1.bw ../data_for_workshop/bigWig/NAg_Rep2.bw ../data_for_workshop/bigWig/TAg_Rep1.bw ../data_for_workshop/bigWig/TAg_Rep2.bw


The options used here are the same that we used above for the density plots. The only changes we have made are the input file which here is the bed files of specific sites that we made earlier

In [ ]:
plotProfile -m NAg_specific_sites_AvgProfile.matrix --outFileName NAg_specific_sites_AvgProfile.pdf --perGroup --colors blue lightblue red pink
plotProfile -m TAg_specific_sites_AvgProfile.matrix --outFileName TAg_specific_sites_AvgProfile.pdf --perGroup --colors blue lightblue red pink

Here is what each of these options do:
* <span style="color:#0000DD">-m NAg_specific_sites_AvgProfile.matrix</span> - The input matrix file
* <span style="color:#0000DD">--outFileName NAg_specific_sites_AvgProfile.pdf</span> - The output file. Similar to the plotHeatmap function multiple file formats can be used (png, svg, tiff etc.)
* <span style="color:#0000DD">--perGroup</span> - Show all average profiles in a single plot. Default is to show a separate plot per sample
* <span style="color:#0000DD">--colors</span> - What colors to use for the lines per sample. This option will accept both names of colors and hex values (e.g. #0000DD)